<a href="https://colab.research.google.com/github/Mongeese1/phyiscs-big-data/blob/master/gravewaves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math as math
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy as sp
import scipy.special as special
from scipy.misc import derivative
import pandas as pd
import sympy as sym
from scipy.optimize import fmin
e=math.e
pi=math.pi

In [9]:
f = 1e-4
G = 6.67*10**(-11)
c = 3*10**8
neg=1j
# convert 10**30 kg to natural numbers
m_one = 10**30/(c**2/G)
# print(m_one)
m_two = 15**30/(c**2/G)
# convert 356 ly to natural numbers
# A = 356*24*60*60
# A = np.sqrt()
M = m_one+m_two
S_O = 6*10**(-49)
f_s = 20
f_0 = 70
l = f/f_0
t = 10
phic = 100
v=(pi*M*f)**(1/3)
n = m_one*m_two/(M**2) 
p = [phic, t, M, n]
low = 1
high = 10
# p = np.array(p)

a = [1, 0, (20/9*(743/336 + n*11/4)), -16*pi, 10*(3058673/1016064 + n*5429/1008 + (n**2) * 617/144)]
# a = np.array(a)
# zum = []
def Sum(z,a):
  z *=2
  j=0
  for i in range(len(a)):
    j += a[i]*(v**i)
  return j

def S(y,S_O):
  S = S_O*(y**(-4.14)) - 5*(y**2) + (111*(1-y**2 + ((y**4)/2))/(1+(y**2)/2))
  return S
# 
zum = Sum(1,a)

# psif = 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)

# waveform = A*(f**(-7/6))*(e**(neg*psif))

def waveform1(f,phic,t,M,n,zum,v,A):
  # phic, t, M, n = p
  psif = 2*pi * f * t - phic - pi/4 + (3/(128*n*(v**5)))*(zum)
  waveform = A*(f**(-7/6))*(e**(neg*psif))
  return waveform
  

y = sym.symbols('y')
indefa = sym.integrate((f**(-7/6))/S(f,S_O),y)

defa = indefa.replace(y,f_0) - indefa.replace(y,f_s)

defa = float(defa)
A = np.sqrt(10 / 4 * defa)

print(A)
  
# print(fora)
# def deriv(x):
# def deriv(x):
#   phic, t, M , n = sym.symbols("phic t M n")
#   return sym.diff(waveform, phic)

# phic, t, M , n = sym.symbols("phic t M n", real=False)
# sym.diff(waveform, t)

def deriv(num,p,phic,t,M,n,zum,v):
  if num == 0:
    dphic = p[0]/1000
    pphic = p.copy()
    pphic[num] += dphic
    dy = waveform1(f,dphic,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dphic)
  if num == 1:
    dt = p[1]/1000
    pt = p.copy()
    pt[num] += dt
    dy = waveform1(f,dt,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dt)
  if num == 2:
    dM = p[2]/1000
    pM = p.copy()
    pM[num] += dM
    dy = waveform1(f,dM,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dM)  
  if num == 3:
    dn = p[3]/1000
    pn = p.copy()
    pn[num] += dn
    dy = waveform1(f,dn,t,M,n,zum,v,A) - waveform1(f,phic,t,M,n,zum,v,A)
    return(dy/dn)
  else:
    print("Function accepts values of 0-3, where these values are which derivitave is being taken, 0=phic, 1=t, 2=M, 3=n")
    return
# deriv(0,p,phic,t,M,n,zum,v)



def create_fisher_matrix(increment,p,phic,t,M,n,zum,v,num,S_O):
    fisher_matrix = []
    for i in range(len(p)):
        fisher_matrix.append([])
        for j in range(len(p)):
            integral = 0
            for k in range(int(f_s*(1/increment)), int(f_0*(1/increment))):
                integral += np.real(deriv(i,p,phic,t,M,n,zum,v)*deriv(j,p,phic,t,M,n,zum,v)/S(k*increment,S_O))
            # print(integral)
            fisher_matrix[-1].append(4 * integral)
    return(fisher_matrix)

for i in range(4):
  matrix = create_fisher_matrix(0.01,p,phic,t,M,n,zum,v,i,S_O)
print(matrix)



inverse = np.linalg.inv(matrix)
print(inverse)

diag = np.sqrt(np.diag(inverse))
# print(np.sqrt(diag))

print(diag)

228.62665052496337
[[-91357998486071.62, -1010735007554722.0, 1229228537.004651, -1.9497832781493093e+21], [-1010735007554722.0, -1.0615388447497006e+16, 10380738635.60631, -2.038553818802831e+22], [1229228537.004651, 10380738635.60631, 1738.5691044168223, 1.9500871987799348e+16], [-1.9497832781493093e+21, -2.038553818802831e+22, 1.9500871987799348e+16, -3.913205474324316e+28]]
[[ 3.25585478e+00  8.14595556e-01 -8.89817112e+04 -6.30925238e-07]
 [ 7.25162977e-01  3.67368169e-01 -2.34200975e+04 -2.39180408e-07]
 [-8.72494128e+04 -2.54308571e+04  2.45426703e+09  1.88183193e-02]
 [-5.83472770e-07 -2.44638483e-07  1.78571429e-02  1.65413304e-13]]
[1.80439873e+00 6.06109041e-01 4.95405594e+04 4.06710344e-07]
